In [1]:
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({"font.size":16})
import os
import sys

sys.path.append(os.path.abspath('../..'))
from modules import io
import modules.vascular_data as sv
from scipy.interpolate import Rbf
import vtk

In [2]:
#CASE = "0110"

CASE = "0144"

gen  = 3

#CSVS = [
#    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/aorta/avg/",
#    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/right_iliac/avg/",
#]

CSVS = [
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/aorta/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/celiac_hepatic/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/celiac_splenic/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/ext_iliac_left/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/renal_left/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/renal_right/avg/",
    "/media/marsdenlab/Data1/UQ/"+CASE+"/sim_wom_rcr_real/"+str(gen)+"/SMA/avg/"
]

FIG_DIR  = "/media/marsdenlab/Data1/UQ/"+CASE+"/figures/3dplots"

BASE_VTU = "/media/marsdenlab/Data1/UQ/"+CASE+"/generations/\
1/coarse/0/exterior.vtp"

io.mkdir(FIG_DIR)

quants = [
    "area",
    "radius_actual",
    "pressure_0",
    "vWSS_3_boundary",
    "velocity_3"
]

dfs_mean = []
dfs_std  = []

for c in CSVS:
    dstd = pd.read_csv(c+'std.csv')
    davg = pd.read_csv(c+'mean.csv')
    
    dfs_std.append(dstd)
    dfs_mean.append(davg)
    
df_mean = pd.concat(dfs_mean)
df_std = pd.concat(dfs_std)

In [3]:
vtu = sv.vtk_read_native_polydata(BASE_VTU)

mu  = df_mean.groupby(["path","point"]).mean()
std = df_std.groupby(["path","point"]).mean()
print(mu)
print(std)

                   Unnamed: 0      area  generation     length        nx  \
path        point                                                          
SMA         0         71216.5  3.049998         3.0   6.236725 -0.292789   
            1         71257.5  2.723842         3.0   5.993853 -0.278528   
            2         71298.5  0.849015         3.0   4.755960 -0.237956   
            3         71339.5  0.308080         3.0   2.615359 -0.128520   
            4         71380.5  0.274140         3.0   1.864211 -0.000669   
            5         71421.5  0.287426         3.0   1.909994  0.032587   
            6         71462.5  0.305448         3.0   1.967565  0.028290   
            7         71503.5  0.337746         3.0   2.068865  0.019194   
            8         71544.5  0.336763         3.0   2.065847  0.056763   
            9         71585.5  0.363363         3.0   2.145044  0.162007   
            10        71626.5  0.374382         3.0   2.176923  0.223285   
            

In [4]:
S1 = 5
S2 = -5
EPS = 1e-4

x = mu['x'].values[S1:S2]
y = mu['y'].values[S1:S2]
z = mu['z'].values[S1:S2]

for q in quants:
    print(q)
    v1 = std[q].values[S1:S2]
    v2 = v1/(mu[q].values[S1:S2]+EPS)
    v3 = (v1-np.amin(v1))/(np.amax(v1)-np.amin(v1)+EPS)
    
    interp1 = Rbf(x,y,z,v1, function='linear')
    interp2 = Rbf(x,y,z,v2, function='linear')
    interp3 = Rbf(x,y,z,v3, function='linear')

    for interp,label in zip([interp1,interp2,interp3], ['_std','_std_relative','_std_norm']):
        array = vtk.vtkDoubleArray()
        array.SetNumberOfComponents(1)
        array.SetNumberOfTuples(vtu.GetNumberOfPoints())
        for i in range(vtu.GetNumberOfPoints()):
            point = vtu.GetPoint(i)
            v = interp(*point)
            array.SetValue(i, v)

        vtu.GetPointData().AddArray(array)
        array.SetName(q+label)

sv.vtk_write_native_polydata(vtu,FIG_DIR+'/{}.vtp'.format(gen))

area
radius_actual
pressure_0
vWSS_3_boundary
velocity_3
